In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
warnings.filterwarnings("ignore")
import tensorflow as tf
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

In [65]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
oil=pd.read_csv("oil.csv")
transaction=pd.read_csv("transactions.csv")
holidays_events=pd.read_csv("holidays_events.csv")
stores=pd.read_csv("stores.csv")


In [77]:
train

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [95]:
combined_data = pd.concat([train, test], ignore_index=True)
combined_data["index"]=combined_data.index
combined_data= combined_data.set_index(['store_nbr','family','date']).sort_index()

In [96]:
combined_data

id  sales  onpromotion    index
store_nbr family     date                                            
1         AUTOMOTIVE 2013-01-01        0    0.0            0        0
                     2013-01-02     1782    2.0            0     1782
                     2013-01-03     3564    3.0            0     3564
                     2013-01-04     5346    3.0            0     5346
                     2013-01-05     7128    5.0            0     7128
...                                  ...    ...          ...      ...
54        SEAFOOD    2017-08-27  3022139    NaN            0  3022139
                     2017-08-28  3023921    NaN            0  3023921
                     2017-08-29  3025703    NaN            0  3025703
                     2017-08-30  3027485    NaN            0  3027485
                     2017-08-31  3029267    NaN            0  3029267

[3029400 rows x 4 columns]

In [97]:
combined_data['lag1'] = combined_data.groupby(['family',"store_nbr"])['sales'].shift(364, fill_value=0)
combined_data['lag2'] = combined_data.groupby(['family',"store_nbr"])['sales'].shift(728, fill_value=0)
combined_data.reset_index(inplace=True)

In [99]:
train_lagged = combined_data[combined_data["date"] <= "2017-08-15"]
test_lagged =combined_data[combined_data["date"] > "2017-08-15"]

In [102]:
train_lagged.index=train_lagged["index"]
test_lagged.index=test_lagged["index"]

In [104]:
train=train_lagged.sort_index()
test=test_lagged.sort_index()

In [109]:
test=test.drop(["sales"],axis=1)

In [111]:
oil.rename(columns={"dcoilwtico":"oil price"},inplace=True)
train=pd.merge(train,oil,on="date", how="left")
test=pd.merge(test,oil,on="date",how="left")

In [112]:
train=pd.merge(train,stores,on="store_nbr",how="left")
test=pd.merge(test,stores,on="store_nbr",how="left")

In [113]:
condition=(holidays_events.transferred==True)|(holidays_events.type=="Word Day")
special_days=holidays_events.drop(holidays_events[condition].index)
special_days=special_days.drop(["type","description","transferred"],axis=1)
special_days["special days"]=1
special_days=special_days.drop(special_days[special_days.date.duplicated()].index)

national=special_days[special_days.locale=="National"]
local=special_days[special_days.locale=="Local"]
regional=special_days[special_days.locale=="Regional"]

national=national.drop(["locale","locale_name"],axis=1)
train=pd.merge(train,national,on="date",how="left")
test=pd.merge(test,national,on="date",how="left")

for i,j in zip(regional.date,regional.locale_name):
    train["special days"][(train.date==i) & (train.state==j)]=1
for i,j in zip(regional.date,regional.locale_name):
    test["special days"][(train.date==i) & (train.state==j)]=1

for i,j in zip(local.date,local.locale_name):
    train["special days"][(train.date==i) & (train.city==j)]=1
for i,j in zip(local.date,local.locale_name):
    test["special days"][(train.date==i) & (train.city==j)]=1

In [114]:
train.head(5)

,store_nbr,family,date,id,sales,onpromotion,index,lag1,lag2,oil price,city,state,type,cluster,special days
0,1,AUTOMOTIVE,2013-01-01,0,0.0,0,0,0.0,0.0,NaN,Quito,Pichincha,D,13,1.0
1,1,BABY CARE,2013-01-01,1,0.0,0,1,0.0,0.0,NaN,Quito,Pichincha,D,13,1.0
2,1,BEAUTY,2013-01-01,2,0.0,0,2,0.0,0.0,NaN,Quito,Pichincha,D,13,1.0
3,1,BEVERAGES,2013-01-01,3,0.0,0,3,0.0,0.0,NaN,Quito,Pichincha,D,13,1.0
4,1,BOOKS,2013-01-01,4,0.0,0,4,0.0,0.0,NaN,Quito,Pichincha,D,13,1.0


In [115]:
train["oil price"]=train["oil price"].fillna(method="bfill")
test["oil price"]=test["oil price"].fillna(method="bfill")

In [116]:
train.head(5)

,store_nbr,family,date,id,sales,onpromotion,index,lag1,lag2,oil price,city,state,type,cluster,special days
0,1,AUTOMOTIVE,2013-01-01,0,0.0,0,0,0.0,0.0,93.14,Quito,Pichincha,D,13,1.0
1,1,BABY CARE,2013-01-01,1,0.0,0,1,0.0,0.0,93.14,Quito,Pichincha,D,13,1.0
2,1,BEAUTY,2013-01-01,2,0.0,0,2,0.0,0.0,93.14,Quito,Pichincha,D,13,1.0
3,1,BEVERAGES,2013-01-01,3,0.0,0,3,0.0,0.0,93.14,Quito,Pichincha,D,13,1.0
4,1,BOOKS,2013-01-01,4,0.0,0,4,0.0,0.0,93.14,Quito,Pichincha,D,13,1.0


In [117]:
train["special days"]=train["special days"].fillna(0)
test["special days"]=test["special days"].fillna(0)

In [118]:
train=train.set_index("id")
test=test.set_index("id")

In [119]:
train=train.drop(["city","state","cluster"],axis=1)
test=test.drop(["city","state","cluster"],axis=1)


In [120]:
train.head(5)

,store_nbr,family,date,sales,onpromotion,index,lag1,lag2,oil price,type,special days
id,,,,,,,,,,,
0,1,AUTOMOTIVE,2013-01-01,0.0,0,0,0.0,0.0,93.14,D,1.0
1,1,BABY CARE,2013-01-01,0.0,0,1,0.0,0.0,93.14,D,1.0
2,1,BEAUTY,2013-01-01,0.0,0,2,0.0,0.0,93.14,D,1.0
3,1,BEVERAGES,2013-01-01,0.0,0,3,0.0,0.0,93.14,D,1.0
4,1,BOOKS,2013-01-01,0.0,0,4,0.0,0.0,93.14,D,1.0


In [121]:
train["family"].unique()

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

In [122]:
train["type"].unique()

array(['D', 'C', 'B', 'E', 'A'], dtype=object)

In [123]:
def convert(df,L,name):
    dic={}
    for j in range (0,len(L)):
        dic[L[j]]=j
    df[name]=df[name].replace(dic)

In [124]:
L=['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD']
convert(test,L,"family")
convert(train,L,"family")

L=["A","B","C","D","E"]
convert(test,L,"type")
convert(train,L,"type")



In [125]:
train

,store_nbr,family,date,sales,onpromotion,index,lag1,lag2,oil price,type,special days
id,,,,,,,,,,,
0,1,0,2013-01-01,0.000,0,0,0.000,0.000,93.14,3,1.0
1,1,1,2013-01-01,0.000,0,1,0.000,0.000,93.14,3,1.0
2,1,2,2013-01-01,0.000,0,2,0.000,0.000,93.14,3,1.0
3,1,3,2013-01-01,0.000,0,3,0.000,0.000,93.14,3,1.0
4,1,4,2013-01-01,0.000,0,4,0.000,0.000,93.14,3,1.0
...,...,...,...,...,...,...,...,...,...,...,...
3000883,9,28,2017-08-15,438.133,0,3000883,394.643,653.323,47.57,1,0.0
3000884,9,29,2017-08-15,154.553,1,3000884,116.874,100.000,47.57,1,0.0
3000885,9,30,2017-08-15,2419.729,148,3000885,1262.692,2078.707,47.57,1,0.0


81.0

In [126]:
train=train.set_index("date")
train.index=pd.to_datetime(train.index)

In [127]:
def create_features(df):
    df["quarter"]=df.index.quarter
    df["dow"]=df.index.day_of_week
    df["month"]=df.index.month
    df["year"]=df.index.year
    df["dayofyear"]=df.index.dayofyear
    return df
train=create_features(train)

In [128]:
train.head()

,store_nbr,family,sales,onpromotion,index,lag1,lag2,oil price,type,special days,quarter,dow,month,year,dayofyear
date,,,,,,,,,,,,,,,
2013-01-01,1,0,0.0,0,0,0.0,0.0,93.14,3,1.0,1,1,1,2013,1
2013-01-01,1,1,0.0,0,1,0.0,0.0,93.14,3,1.0,1,1,1,2013,1
2013-01-01,1,2,0.0,0,2,0.0,0.0,93.14,3,1.0,1,1,1,2013,1
2013-01-01,1,3,0.0,0,3,0.0,0.0,93.14,3,1.0,1,1,1,2013,1
2013-01-01,1,4,0.0,0,4,0.0,0.0,93.14,3,1.0,1,1,1,2013,1


In [131]:
y_train=train["sales"]
x_train=train.drop(["sales"],axis=1)

In [132]:
test.head(5)

,store_nbr,family,date,onpromotion,index,lag1,lag2,oil price,type,special days
id,,,,,,,,,,
3000888,1,0,2017-08-16,0,3000888,5.0,0.0,46.8,3,0.0
3000889,1,1,2017-08-16,0,3000889,0.0,0.0,46.8,3,0.0
3000890,1,2,2017-08-16,2,3000890,5.0,2.0,46.8,3,0.0
3000891,1,3,2017-08-16,20,3000891,2010.0,2194.0,46.8,3,0.0
3000892,1,4,2017-08-16,0,3000892,0.0,0.0,46.8,3,0.0


In [133]:
size=int(0.9*len(x_train))
xtrain=x_train[0:size]
ytrain=y_train[0:size]
xtest=x_train[size:]
ytest=y_train[size:]

In [134]:
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [135]:
# xgb = xgb.XGBRegressor(random_state=35643419,tree_method='hist',device='cuda')
# xgb.set_params(predictor='gpu_predictor')
# xgb.fit(x_train,y_train)
reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                           n_estimators=1500,
                           early_stopping_rounds=50,
                           objective="reg:squaredlogerror",
                           max_depth=3,
                           learning_rate=0.01)
reg.fit(xtrain, ytrain,
        eval_set=[(xtrain, ytrain), (xtest, ytest)],
        verbose=100)

[0]	validation_0-rmsle:3.63888	validation_1-rmsle:4.07826
[100]	validation_0-rmsle:3.17022	validation_1-rmsle:3.48381
[200]	validation_0-rmsle:2.78359	validation_1-rmsle:2.94287
[300]	validation_0-rmsle:2.46364	validation_1-rmsle:2.44613
[400]	validation_0-rmsle:2.20624	validation_1-rmsle:2.00386
[500]	validation_0-rmsle:2.01184	validation_1-rmsle:1.62507
[600]	validation_0-rmsle:1.87728	validation_1-rmsle:1.32339
[700]	validation_0-rmsle:1.79424	validation_1-rmsle:1.11070
[800]	validation_0-rmsle:1.74629	validation_1-rmsle:0.97693
[900]	validation_0-rmsle:1.71813	validation_1-rmsle:0.89839
[1000]	validation_0-rmsle:1.69774	validation_1-rmsle:0.85153
[1100]	validation_0-rmsle:1.67950	validation_1-rmsle:0.82503
[1200]	validation_0-rmsle:1.66156	validation_1-rmsle:0.81004
[1300]	validation_0-rmsle:1.64534	validation_1-rmsle:0.79892
[1400]	validation_0-rmsle:1.63060	validation_1-rmsle:0.79037
[1499]	validation_0-rmsle:1.61929	validation_1-rmsle:0.78404


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=50,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1500, n_jobs=None,
             num_parallel_tree=None, objective='reg:squaredlogerror', ...)

In [136]:
save_index=test.index
test=test.set_index("date")
test.index=pd.to_datetime(test.index)
test=create_features(test)

In [137]:
test.head(5)

,store_nbr,family,onpromotion,index,lag1,lag2,oil price,type,special days,quarter,dow,month,year,dayofyear
date,,,,,,,,,,,,,,
2017-08-16,1,0,0,3000888,5.0,0.0,46.8,3,0.0,3,2,8,2017,228
2017-08-16,1,1,0,3000889,0.0,0.0,46.8,3,0.0,3,2,8,2017,228
2017-08-16,1,2,2,3000890,5.0,2.0,46.8,3,0.0,3,2,8,2017,228
2017-08-16,1,3,20,3000891,2010.0,2194.0,46.8,3,0.0,3,2,8,2017,228
2017-08-16,1,4,0,3000892,0.0,0.0,46.8,3,0.0,3,2,8,2017,228


In [138]:
test_predictions = reg.predict(test)
test.index=save_index

In [139]:
submission = pd.DataFrame({'id':test.index, 'sales': test_predictions.flatten()})

In [140]:
submission.to_csv("ok",index=False)

In [141]:
submission

,id,sales
0,3000888,3.553900
1,3000889,0.547810
2,3000890,20.816614
3,3000891,1134.700073
4,3000892,0.503983
...,...,...
28507,3029395,399.652832
28508,3029396,90.956139
28509,3029397,938.613403
28510,3029398,41.641262


In [44]:
test.reset_index(inplace=True)

In [45]:
test.head()

,index,store_nbr,family,onpromotion,oil price,type,special days,lag1,lag2,quarter,dow,month,year,dayofyear
0,3000888,41,28,1,46.8,3,0.0,149.141,144.421,3,2,8,2017,228
1,3000889,42,3,19,46.8,3,0.0,1649.000,0.000,3,2,8,2017,228
2,3000890,42,2,2,46.8,3,0.0,6.000,0.000,3,2,8,2017,228
3,3000891,42,1,0,46.8,3,0.0,0.000,0.000,3,2,8,2017,228
4,3000892,42,0,0,46.8,3,0.0,3.000,0.000,3,2,8,2017,228
